# Demonstration of Givens rotation decomposition

In [1]:
import numpy
from openfermion.utils import givens_decomposition
from scipy.linalg import qr

numpy.set_printoptions(precision=3, linewidth=150, suppress=True)

A Slater determinant can be represented by an $m \times n$ matrix with orthonormal rows. Let's obtain such a matrix $Q$ using Scipy's QR decomposition algorithm.

In [2]:
m, n = (3, 6)

# Obtain a random matrix of orthonormal rows
x = numpy.random.randn(n, n)
y = numpy.random.randn(n, n)
A = x + 1j*y
Q, R = qr(A)
Q = Q[:m, :]
print(Q)

[[-0.006+0.277j -0.197-0.123j  0.375+0.307j  0.267-0.403j -0.035-0.016j  0.515+0.366j]
 [ 0.003-0.429j -0.742+0.077j  0.055+0.372j -0.123+0.276j -0.020-0.144j  0.028-0.07j ]
 [-0.278+0.289j -0.299+0.054j -0.322-0.507j -0.033+0.158j  0.142-0.476j  0.259+0.215j]]


There exist unitary matrices $V$ and $U$ such that $VQU^\dagger$ is an $m \times n$ diagonal matrix. The matrix $VQ$ represents the same Slater determinant as $Q$ up to an overall phase (equal to $\det(V)$), but it has zeros in the upper diagonal, which saves some work in the preparation of the Slater determinant. The unitary $U$ can be written in the form

$$U = G_{N_G} \cdots G_2 G_1$$

where the $G_k$ are complex Givens rotations. We can obtain $V$, the $G_k$, and the diagonal entries of $VQU^\dagger$ using the `givens_decomposition` function.

In [3]:
# Get Givens decomposition of U
V, givens_rotations, diagonal = givens_decomposition(Q)
print(V)
print()
print(diagonal)

[[ 0.094-0.149j  0.242-0.911j  0.056+0.28j ]
 [ 0.092+0.436j  0.024-0.317j -0.164-0.82j ]
 [ 0.878+0.j    -0.025+0.102j  0.466-0.035j]]

[-0.953-0.303j  0.132+0.991j -0.200+0.98j ]


We didn't print the Givens rotations in the above block because it wouldn't look pretty. The Givens rotations are returned as a list of tuples of tuples. We will now iterate through the tuples of the list, and print the innermost tuple within each tuple as a string.

In [4]:
for parallel_set in givens_rotations:
    print(tuple(["{}, {}, {:.3f}, {:.3f}".format(i, j, theta, phi)
                 for i, j, theta, phi in parallel_set]))

('2, 3, 0.297, -2.332',)
('1, 2, 0.735, 1.110', '3, 4, 0.662, -2.365')
('0, 1, 1.084, 1.537', '2, 3, 0.873, -1.708', '4, 5, 1.054, 2.647')
('1, 2, 1.349, 2.749', '3, 4, 1.219, 0.577')
('2, 3, 1.079, -1.236',)


There are 5 tuples printed, and within each tuple, there are strings of the form 'i, j, theta, phi'. Each such string represents an innermost tuple, and it is a description of a complex Givens rotation of the coordinates $i$ and $j$ by angles $\theta$ and $\varphi$. The $2 \times 2$ matrix corresponding to this rotation is

$$
\begin{pmatrix}
\cos \theta & -e^{i \varphi} \sin \theta \\
\sin \theta & e^{i \varphi} \cos \theta
\end{pmatrix}.
$$

The fact that there are 5 tuples means that the circuit depth to prepare the Slater determinant corresponding to $Q$ (up to a phase) has depth 5. All of the rotations within the tuple can be performed in parallel; this is possible because the indices to be rotated are disjoint. For instance, in the third step, we can perform three rotations simultaneously, on coordinates $(0, 1)$, $(2, 3)$, and $(4, 5)$.

Let's check that $VQ$ has zeros in the upper right corner.

In [5]:
# Check that VQ has zeros in upper right corner
W = V.dot(Q)
print(W)

[[-0.446-0.142j -0.177+0.631j  0.557-0.105j  0.141+0.101j  0.000+0.j    -0.000-0.j   ]
 [ 0.026+0.193j  0.135+0.376j -0.343+0.53j   0.420+0.127j -0.456-0.052j -0.000+0.j   ]
 [-0.081+0.399j -0.300-0.15j   0.122+0.04j   0.200-0.298j  0.034-0.239j  0.587+0.417j]]


Now let's check the Givens decomposition. For each set of Givens rotations that can be performed in parallel, we construct the matrices corresponding to the Givens rotations and multiply them together. Then, we multiply $W = VQ$ repeatedly on the right by these matrices and check that the correct elements are zeroed out.

In [6]:
# Check the Givens decomposition
def expanded_givens(G, i, j, n):
    expanded_G = numpy.eye(n, dtype=complex)
    expanded_G[([i], [j]), (i, j)] = G
    return expanded_G

U = numpy.eye(n, dtype=complex)
for parallel_set in givens_rotations:
    print("Number of rotations to perform in parallel: {}".format(len(parallel_set)))
    combined_givens = numpy.eye(n, dtype=complex)
    for i, j, theta, phi in parallel_set:
        c = numpy.cos(theta)
        s = numpy.sin(theta)
        phase = numpy.exp(1.j * phi)
        G = numpy.array([[c, -phase * s],
                     [s, phase * c]], dtype=complex)
        expanded_G = expanded_givens(G, i, j, n)
        combined_givens = combined_givens.dot(expanded_G)
    W = W.dot(combined_givens.T.conj())
    U = combined_givens.dot(U)
    print(W)
    print()

Number of rotations to perform in parallel: 1
[[-0.446-0.142j -0.177+0.631j  0.582-0.11j  -0.000+0.j     0.000+0.j    -0.000-0.j   ]
 [ 0.026+0.193j  0.135+0.376j -0.217+0.444j -0.465+0.363j -0.456-0.052j -0.000+0.j   ]
 [-0.081+0.399j -0.300-0.15j   0.094-0.064j  0.111+0.347j  0.034-0.239j  0.587+0.417j]]

Number of rotations to perform in parallel: 2
[[-0.446-0.142j -0.239+0.851j -0.000-0.j    -0.000+0.j    -0.000+0.j    -0.000-0.j   ]
 [ 0.026+0.193j -0.101+0.017j  0.314+0.543j -0.589+0.46j   0.000+0.j    -0.000+0.j   ]
 [-0.081+0.399j -0.212-0.036j -0.213-0.184j -0.000+0.154j  0.181+0.367j  0.587+0.417j]]

Number of rotations to perform in parallel: 3
[[-0.953-0.303j -0.000-0.j    -0.000-0.j    -0.000-0.j    -0.000-0.j    -0.000+0.j   ]
 [-0.000-0.j     0.029+0.218j  0.489+0.844j -0.000-0.j    -0.000+0.j     0.000+0.j   ]
 [-0.000-0.j    -0.092+0.451j -0.020-0.102j -0.261-0.155j  0.366+0.742j  0.000-0.j   ]]

Number of rotations to perform in parallel: 2
[[-0.953-0.303j -0.000-0.j 

Finally, let's check that the final matrix, $VQU^\dagger$, is indeed diagonal, and that its entries match the ones returned by the function.

In [7]:
# Check the diagonal entries
D = numpy.zeros((m, n), dtype=complex)
D[numpy.diag_indices(m)] = diagonal
print("V * Q * U^\dagger matches the returned diagonal:")
print(numpy.all(numpy.isclose(D, V.dot(Q.dot(U.T.conj())))))

V * Q * U^\dagger matches the returned diagonal:
True
